# Using Geographical and Temporal Weighted Regression to Explore the Spatial Vairation of COVID-19 in the Contiguous United States

Weiye Chen, Shaohua Wang, University of Illinois, Urbana-Champaign

In [1]:
from mgwr.gwr import GWR, MGWR
from mgwr.sel_bw import Sel_BW
import pandas as pd
import geopandas as gpd
import plotly.graph_objects as go
import numpy as np
from scipy import stats
# from sklearn.linear_model import LinearRegression
# from sklearn.feature_selection import SelectFromModel
from statsmodels.api import OLS

from IPython.display import HTML

%matplotlib inline
%config InlineBackend.figure_formats = ['svg']

## Data Preparation

### Population Census Data

Retrieving the county-level census data in the United States incorporated in a shapefile.

In [2]:
!cd ..
!wget -nc https://s3-eu-west-1.amazonaws.com/pfigshare-u-files/22153815/USA_Counties_as_Shape.zip

File ‘USA_Counties_as_Shape.zip’ already there; not retrieving.



In [3]:
counties = gpd.read_file("zip://USA_Counties_as_Shape.zip")
counties['FIPS'] = counties['FIPS'].astype('int')
counties = counties.set_index('FIPS')

### COVID-19 Data

The data is retrieved from [Johns Hopkins CSSE COVID-19 cases dataset repository](https://github.com/CSSEGISandData/COVID-19/).

In [4]:
confirmed_cases = pd.read_csv(
    "https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv"
)
# confirmed_cases = confirmed_cases[confirmed_cases['Province_State'] == 'New York']
confirmed_cases = confirmed_cases[confirmed_cases['Country_Region'] == 'US']
confirmed_cases = confirmed_cases[confirmed_cases['Province_State'] != 'Alaska']
confirmed_cases = confirmed_cases[confirmed_cases['Province_State'] != 'Hawaii']
confirmed_cases = confirmed_cases[confirmed_cases['UID'] != 84046102] # South Dokata...
confirmed_cases = confirmed_cases[confirmed_cases['Admin2'].isnull() == False]
confirmed_cases = confirmed_cases[confirmed_cases['FIPS'] < 80000]
confirmed_cases['FIPS'] = confirmed_cases['FIPS'].astype('int')
confirmed_cases = confirmed_cases.set_index('FIPS')
confirmed_cases.head()

,UID,iso2,iso3,code3,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,...,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,84001001,US,USA,840,Autauga,Alabama,US,32.539527,-86.644082,"Autauga, Alabama, US",...,8,10,12,12,12,12,12,12,15,17
1003,84001003,US,USA,840,Baldwin,Alabama,US,30.727750,-87.722071,"Baldwin, Alabama, US",...,20,24,28,29,29,38,42,44,56,59
1005,84001005,US,USA,840,Barbour,Alabama,US,31.868263,-85.387129,"Barbour, Alabama, US",...,0,0,1,2,2,2,3,3,4,9
1007,84001007,US,USA,840,Bibb,Alabama,US,32.996421,-87.125115,"Bibb, Alabama, US",...,3,4,4,4,5,7,8,9,9,11
1009,84001009,US,USA,840,Blount,Alabama,US,33.982109,-86.567906,"Blount, Alabama, US",...,5,6,9,10,10,10,10,10,11,12


In [5]:
cases_ny = confirmed_cases.join(counties, rsuffix = ' ')
cases_ny.columns.values

array(['UID', 'iso2', 'iso3', 'code3', 'Admin2', 'Province_State',
       'Country_Region', 'Lat', 'Long_', 'Combined_Key', '1/22/20',
       '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20', '1/28/20',
       '1/29/20', '1/30/20', '1/31/20', '2/1/20', '2/2/20', '2/3/20',
       '2/4/20', '2/5/20', '2/6/20', '2/7/20', '2/8/20', '2/9/20',
       '2/10/20', '2/11/20', '2/12/20', '2/13/20', '2/14/20', '2/15/20',
       '2/16/20', '2/17/20', '2/18/20', '2/19/20', '2/20/20', '2/21/20',
       '2/22/20', '2/23/20', '2/24/20', '2/25/20', '2/26/20', '2/27/20',
       '2/28/20', '2/29/20', '3/1/20', '3/2/20', '3/3/20', '3/4/20',
       '3/5/20', '3/6/20', '3/7/20', '3/8/20', '3/9/20', '3/10/20',
       '3/11/20', '3/12/20', '3/13/20', '3/14/20', '3/15/20', '3/16/20',
       '3/17/20', '3/18/20', '3/19/20', '3/20/20', '3/21/20', '3/22/20',
       '3/23/20', '3/24/20', '3/25/20', '3/26/20', '3/27/20', '3/28/20',
       '3/29/20', '3/30/20', '3/31/20', '4/1/20', '4/2/20', '4/3/20',
       '4/

## Linear Model: Selecting Variables from Candidates

In [6]:
variable_names = cases_ny.columns[-51:-4]

In [7]:
variable_names

Index(['POP2010', 'POP10_SQMI', 'POP2012', 'POP12_SQMI', 'WHITE', 'BLACK',
       'AMERI_ES', 'ASIAN', 'HAWN_PI', 'HISPANIC', 'OTHER', 'MULT_RACE',
       'MALES', 'FEMALES', 'AGE_UNDER5', 'AGE_5_9', 'AGE_10_14', 'AGE_15_19',
       'AGE_20_24', 'AGE_25_34', 'AGE_35_44', 'AGE_45_54', 'AGE_55_64',
       'AGE_65_74', 'AGE_75_84', 'AGE_85_UP', 'MED_AGE', 'MED_AGE_M',
       'MED_AGE_F', 'HOUSEHOLDS', 'AVE_HH_SZ', 'HSEHLD_1_M', 'HSEHLD_1_F',
       'MARHH_CHD', 'MARHH_NO_C', 'MHH_CHILD', 'FHH_CHILD', 'FAMILIES',
       'AVE_FAM_SZ', 'HSE_UNITS', 'VACANT', 'OWNER_OCC', 'RENTER_OCC',
       'NO_FARMS07', 'AVG_SIZE07', 'CROP_ACR07', 'AVG_SALE07'],
      dtype='object')

The lastest date of report for the confirmed cases of COVID-19:

In [8]:
latest_date = confirmed_cases.columns[-1]
print(latest_date)

4/10/20


In [17]:
X = cases_ny[variable_names]
y = cases_ny[latest_date]
u = cases_ny['Long_']
v = cases_ny['Lat']

In [18]:
X['ELDER'] = X['AGE_85_UP'] + X['AGE_75_84'] + X['AGE_65_74']
X = X.drop([
    'POP10_SQMI', 'POP2010', 'AGE_85_UP', 'AGE_75_84', 'AGE_65_74', 'AGE_55_64', 'AGE_45_54', 
    'AGE_35_44', 'AGE_25_34', 'AGE_20_24', 'AGE_15_19', 'AGE_10_14', 'AGE_5_9', 'AGE_UNDER5'
], axis=1)
X

/Users/weiye/opt/anaconda3/envs/GIS37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,POP2012,POP12_SQMI,WHITE,BLACK,AMERI_ES,ASIAN,HAWN_PI,HISPANIC,OTHER,MULT_RACE,...,AVE_FAM_SZ,HSE_UNITS,VACANT,OWNER_OCC,RENTER_OCC,NO_FARMS07,AVG_SIZE07,CROP_ACR07,AVG_SALE07,ELDER
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,55939,92.594309,42855,9643,232,474,32,1310,466,869,...,3.13,22135,1914,15248,4973,415.0,266.0,42349.0,40.41,6546
1003,190116,115.901069,156153,17105,1216,1348,89,7992,3631,2723,...,2.93,104061,30881,53071,20109,1139.0,167.0,103036.0,88.09,30568
1005,27310,30.193811,13180,12875,114,107,29,1387,894,258,...,3.01,11829,2009,6556,3264,623.0,320.0,56934.0,114.63,3909
1007,23106,36.899933,17381,5047,64,22,13,406,185,203,...,3.09,8981,1028,6011,1942,211.0,181.0,8619.0,-99.00,2906
1009,58107,89.308824,53068,761,307,117,38,4626,2347,684,...,3.07,23887,2309,17384,4194,1414.0,107.0,46735.0,113.33,8439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56037,45509,4.337863,38748,438,423,336,42,6689,2799,1020,...,3.09,18735,2260,11872,4603,244.0,6092.0,46144.0,59.45,3643
56039,22666,5.374937,18821,49,111,235,15,3191,1715,348,...,2.89,12813,3840,5083,3890,180.0,294.0,18474.0,50.93,2098
56041,21533,10.314864,19514,55,168,61,36,1855,860,424,...,3.19,8713,1045,5759,1909,344.0,2159.0,70210.0,78.67,1874


### A trial run of linear regression over all variables

In [19]:
OLS(y,X).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                4/10/20   R-squared (uncentered):                   0.745
Model:                            OLS   Adj. R-squared (uncentered):              0.742
Method:                 Least Squares   F-statistic:                              289.7
Date:                Fri, 10 Apr 2020   Prob (F-statistic):                        0.00
Time:                        20:37:23   Log-Likelihood:                         -25687.
No. Observations:                3107   AIC:                                  5.144e+04
Df Residuals:                    3076   BIC:                                  5.162e+04
Df Model:                          31                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
POP2012       -0.1145      0.009    -13.051      0.000      -0.132      -0.097
POP12_SQMI     0.8420      0.020     42.928      0.000       0.804       0.880
WHITE          0.0162      0.005      3.321      0.001       0.007       0.026
BLACK          0.0238      0.005      4.802      0.000       0.014       0.033
AMERI_ES       0.0079      0.008      1.042      0.297      -0.007       0.023
ASIAN         -0.0072      0.005     -1.334      0.182      -0.018       0.003
HAWN_PI        0.1385      0.041      3.354      0.001       0.058       0.219
HISPANIC       0.0107      0.001     10.608      0.000       0.009       0.013
OTHER          0.0004      0.004      0.085      0.932      -0.008       0.009
MULT_RACE      0.0658      0.015      4.248      0.000       0.035       0.096
MALES          0.0743      0.013      5.816      0.000       0.049       0.099
FEMALES        0.1710      0.015     11.139      0.000       0.141       0.201
MED_AGE      128.1147     93.029      1.377      0.169     -54.291     310.521
MED_AGE_M   -109.8392     54.033     -2.033      0.042    -215.784      -3.894
MED_AGE_F    -10.3346     42.878     -0.241      0.810     -94.408      73.739
HOUSEHOLDS    -0.0978      0.012     -7.894      0.000      -0.122      -0.074
AVE_HH_SZ   1700.2280    298.054      5.704      0.000    1115.823    2284.633
HSEHLD_1_M     0.4089      0.044      9.309      0.000       0.323       0.495
HSEHLD_1_F     0.2678      0.036      7.458      0.000       0.197       0.338
MARHH_CHD      0.1580      0.036      4.419      0.000       0.088       0.228
MARHH_NO_C    -0.0298      0.040     -0.740      0.459      -0.109       0.049
MHH_CHILD      0.3434      0.085      4.044      0.000       0.177       0.510
FHH_CHILD     -0.4476      0.041    -10.907      0.000      -0.528      -0.367
FAMILIES       0.0396      0.035      1.122      0.262      -0.030       0.109
AVE_FAM_SZ -1605.9246    272.222     -5.899      0.000   -2139.679   -1072.170
HSE_UNITS     -0.0385      0.006     -6.161      0.000      -0.051      -0.026
VACANT         0.0593      0.007      8.414      0.000       0.045       0.073
OWNER_OCC     -0.0337      0.007     -4.546      0.000      -0.048      -0.019
RENTER_OCC    -0.0641      0.006    -10.991      0.000      -0.076      -0.053
NO_FARMS07     0.1034      0.038      2.692      0.007       0.028       0.179
AVG_SIZE07     0.0255      0.012      2.208      0.027       0.003       0.048
CROP_ACR07 -4.437e-05      0.000     -0.309      0.758      -0.000       0.000
AVG_SALE07     0.1657      0.096      1.724      0.085      -0.023       0.354
ELDER         -0.0085      0.010     -0.854      0.393      -0.028       0.011
=============================================================

According to the OLS results, we find that the linear model with all variables produces good fit in the model, with a r<sup>2</sup> of 0.745. However, strong multicollinearity exists in this mdoel. Therefore, we need to perform an analysis to select variables.

### Stepwise Regression

#### Forward Selection
Test the linear terms, and sort them in the order of correlation. 

In [20]:
r_squared = []
for i_factor in X.columns:
    r_squared.append((i_factor, OLS(y, X[i_factor]).fit().f_pvalue))

In [21]:
r_squared.sort(key=lambda x:x[1])

In [22]:
r_squared

[('POP12_SQMI', 0.0),
 ('HSEHLD_1_F', 3.4665021999958555e-139),
 ('HSEHLD_1_M', 9.901607061017333e-123),
 ('RENTER_OCC', 2.9289401159682405e-116),
 ('HSE_UNITS', 1.3709204859965177e-90),
 ('HOUSEHOLDS', 3.145800151416882e-90),
 ('ELDER', 3.8348882605157694e-88),
 ('FEMALES', 2.772556694342692e-78),
 ('WHITE', 5.933888869675933e-77),
 ('VACANT', 6.325918968806516e-76),
 ('POP2012', 1.551820124284665e-75),
 ('MARHH_NO_C', 4.099009923704337e-74),
 ('MALES', 6.5933679619569514e-74),
 ('FAMILIES', 2.6752283160453154e-71),
 ('FHH_CHILD', 7.904936986932248e-69),
 ('OWNER_OCC', 2.8703021380528913e-66),
 ('BLACK', 3.8583633012433343e-66),
 ('MARHH_CHD', 2.8514392398037532e-62),
 ('MULT_RACE', 1.6818252802206572e-58),
 ('MHH_CHILD', 1.2493339559434713e-50),
 ('ASIAN', 3.191280053423565e-46),
 ('HISPANIC', 1.7034118727289327e-38),
 ('OTHER', 3.089151226744026e-35),
 ('AMERI_ES', 2.4602051247149515e-17),
 ('HAWN_PI', 4.13041356125312e-12),
 ('AVE_FAM_SZ', 7.233942599471294e-07),
 ('AVE_HH_SZ', 1.8

In [23]:
from scipy.stats import spearmanr

In [24]:
spearmanr(X["POP2012"], y).correlation

0.829542461680131

In [25]:
sp_r = []
for factor in X.columns:
    s = spearmanr(X[factor], y)
    sp_r.append((factor, s.correlation, s.pvalue))

In [26]:
sp_r.sort(key = lambda x: -x[1])
sp_r

[('FHH_CHILD', 0.8402977080932695, 0.0),
 ('FEMALES', 0.8298459834429129, 0.0),
 ('POP2012', 0.829542461680131, 0.0),
 ('MALES', 0.8276727270970254, 0.0),
 ('FAMILIES', 0.8256037951971842, 0.0),
 ('HOUSEHOLDS', 0.8233428267724641, 0.0),
 ('MARHH_CHD', 0.8184640552163794, 0.0),
 ('OWNER_OCC', 0.8182793372664616, 0.0),
 ('RENTER_OCC', 0.8171703385465415, 0.0),
 ('HSE_UNITS', 0.814653729102147, 0.0),
 ('MHH_CHILD', 0.8143471272723334, 0.0),
 ('HSEHLD_1_M', 0.8072034032825226, 0.0),
 ('HSEHLD_1_F', 0.8069873341172963, 0.0),
 ('MARHH_NO_C', 0.8029499291936557, 0.0),
 ('ELDER', 0.7965012876181646, 0.0),
 ('ASIAN', 0.7923742619757498, 0.0),
 ('WHITE', 0.7853203834404263, 0.0),
 ('MULT_RACE', 0.773763599936531, 0.0),
 ('POP12_SQMI', 0.7554261429550923, 0.0),
 ('BLACK', 0.7504873533664838, 0.0),
 ('HAWN_PI', 0.7105996464752383, 0.0),
 ('VACANT', 0.70306580181632, 0.0),
 ('HISPANIC', 0.6751077392350475, 0.0),
 ('OTHER', 0.6642897250592988, 0.0),
 ('AMERI_ES', 0.5985370571228797, 2.07788127096078

#### Backward Elimination

In this section, we use the AIC as a criteria for selecting variables.

In [27]:
selected_vars = []
AIC_low = 1e100
for var, r2 in r_squared:
    vars = selected_vars.copy()
    vars.append(var)
    AIC = OLS(y, X[vars]).fit().aic
    if AIC < AIC_low:
        selected_vars.append(var)
        AIC_low = AIC
        if len(selected_vars) > 20:
            break

In [28]:
print(selected_vars)

['POP12_SQMI', 'HSEHLD_1_F', 'HSEHLD_1_M', 'RENTER_OCC', 'HSE_UNITS', 'HOUSEHOLDS', 'ELDER', 'FEMALES', 'WHITE', 'POP2012', 'MARHH_NO_C', 'MALES', 'FAMILIES', 'FHH_CHILD', 'MARHH_CHD', 'MULT_RACE', 'MHH_CHILD', 'ASIAN', 'HISPANIC', 'OTHER', 'AMERI_ES']


Construct a new data frame with selected variables.

In [29]:
X_new = X[selected_vars]

In [30]:
OLS_results = OLS(y, X_new).fit()

In [31]:
OLS_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                4/10/20   R-squared (uncentered):                   0.739
Model:                            OLS   Adj. R-squared (uncentered):              0.738
Method:                 Least Squares   F-statistic:                              416.7
Date:                Fri, 10 Apr 2020   Prob (F-statistic):                        0.00
Time:                        20:37:27   Log-Likelihood:                         -25721.
No. Observations:                3107   AIC:                                  5.148e+04
Df Residuals:                    3086   BIC:                                  5.161e+04
Df Model:                          21                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
POP12_SQMI     0.8217      0.019     43.414      0.000       0.785       0.859
HSEHLD_1_F     0.2753      0.036      7.747      0.000       0.206       0.345
HSEHLD_1_M     0.4060      0.043      9.504      0.000       0.322       0.490
RENTER_OCC    -0.0273      0.005     -5.609      0.000      -0.037      -0.018
HSE_UNITS      0.0205      0.005      4.267      0.000       0.011       0.030
HOUSEHOLDS    -0.1973      0.026     -7.701      0.000      -0.248      -0.147
ELDER       5.615e-05      0.010      0.006      0.995      -0.019       0.019
FEMALES        0.1852      0.016     11.535      0.000       0.154       0.217
WHITE         -0.0077      0.002     -4.782      0.000      -0.011      -0.005
POP2012       -0.1092      0.009    -12.534      0.000      -0.126      -0.092
MARHH_NO_C    -0.0433      0.039     -1.121      0.262      -0.119       0.032
MALES          0.0927      0.013      7.356      0.000       0.068       0.117
FAMILIES       0.0532      0.034      1.548      0.122      -0.014       0.121
FHH_CHILD     -0.4637      0.040    -11.572      0.000      -0.542      -0.385
MARHH_CHD      0.1669      0.035      4.743      0.000       0.098       0.236
MULT_RACE      0.0665      0.010      6.385      0.000       0.046       0.087
MHH_CHILD      0.3684      0.084      4.405      0.000       0.204       0.532
ASIAN         -0.0327      0.003    -12.225      0.000      -0.038      -0.027
HISPANIC       0.0103      0.001     10.558      0.000       0.008       0.012
OTHER         -0.0243      0.003     -7.873      0.000      -0.030      -0.018
AMERI_ES      -0.0185      0.007     -2.754      0.006      -0.032      -0.005
==============================================================================
Omnibus:                     2589.452   Durbin-Watson:                   1.775
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2207405.505
Skew:                           2.729   Prob(JB):                         0.00
Kurtosis:                     133.466   Cond. No.                     2.75e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.75e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Now we compare the predicted and actual number of confirmed cases.

In [32]:
lr_y_yhat = pd.DataFrame({'Actual # Cases': y, 'Predicted # Cases': OLS_results.predict(X_new)}, index = cases_ny.index)
lr_y_yhat['ratio'] = lr_y_yhat['Predicted # Cases'] / lr_y_yhat['Actual # Cases']

In [33]:
lr_y_yhat['ratio'] = lr_y_yhat['ratio'].replace(np.inf, 1000)
lr_y_yhat[lr_y_yhat['ratio'] <= 0] = 0.4

Intuitively, the model overestimates the number of confirmed cases when the ratio is high, which could suggest those areas were underreporting cases,  not being vulnerable, or doing well in disease control compared to other counties. Likewise, when the ratio is low, the model underestimate the number of confirmed cases, which means the number of confirmed cases is high then usual, and could imply that the areas have potential flaws in handling the outbreak, or other implications. 

In [34]:
from urllib.request import urlopen
import json
#with urlopen('https://raw.githubusercontent.com/cybergis/COVID_19/master/counties_update_new.geojson') as response:
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties1 = json.load(response)

In [71]:
fig = go.Figure(
    go.Choroplethmapbox(
        geojson=counties1, locations=lr_y_yhat.index, 
        z=np.log(lr_y_yhat['ratio']),
        colorscale=[[0, 'blue'], [0.5,'white'], [1,'red']],
        zmid = 0, zauto = True,
        marker_opacity=0.5, marker_line_width=0,
        ids = cases_ny['Combined_Key'],  
        name = 'Predicted / Actual # Cases',
        colorbar_thickness = 10,
        hoverinfo = 'text',
        text = cases_ny['Combined_Key'],
#         showlegend = True,
        showscale = True,
        colorbar = dict(
            title = "Ratio",
            titleside = 'top',
            tickmode = 'array',
            tickvals = np.arange(-10, 10),
            ticktext = np.arange(-10, 10),
            ticks = 'inside',
            outlinewidth = 0
        )
    ))
fig.update_layout(mapbox_style="carto-positron",
                  mapbox_zoom=4, #mapbox_center = {"lat": 37.0902, "lon": -95.7129},)
                  mapbox_center={"lat": 40, "lon": -100},
                 )
fig.update_layout(margin={"r":10,"t":10,"l":10,"b":10})

fig.write_html('viz/lr.html')

## Geographical Weighted Regression

In [134]:
from sklearn.preprocessing import scale
import multiprocessing as mp

In [140]:
X_new_mean = X_new.mean(axis = 0)
X_new_std = X_new.std(axis=0)

In [136]:
coords = list(zip(u,v))
X_new_n = scale(X_new, with_mean=True, with_std=True)
y_n = scale(y, with_mean=True, with_std=True).reshape((-1,1))

In [141]:
#########\
X_new_n = X_new.values
y_n = y.values.reshape((-1,1))

In [142]:
%%time
bw_selector = Sel_BW(coords, y_n, X_new_n)
gwr_bw = bw_selector.search(bw_min=2)
print(gwr_bw)

126.0
CPU times: user 1min 41s, sys: 737 ms, total: 1min 42s
Wall time: 25.8 s


In [143]:
%%time
gwr_model = GWR(coords, y_n, X_new_n, gwr_bw)

CPU times: user 10.3 ms, sys: 1.17 ms, total: 11.5 ms
Wall time: 2.95 ms


In [144]:
print(gwr_model.fit().summary())

Model type                                                         Gaussian
Number of observations:                                                3107
Number of covariates:                                                    22

Global Regression Results
---------------------------------------------------------------------------
Residual sum of squares:                                       2816153982.469
Log-likelihood:                                                  -25718.351
AIC:                                                              51480.702
AICc:                                                             51483.060
BIC:                                                           2816129174.710
R2:                                                                   0.738
Adj. R2:                                                              0.736

Variable                              Est.         SE  t(Est/SE)    p-value
------------------------------- ---------- ---------- --

In [145]:
X_new_n.shape

(3107, 21)

In [146]:
%%time
gwr_results = gwr_model.predict(np.array(coords), X_new_n)

CPU times: user 15.5 s, sys: 130 ms, total: 15.6 s
Wall time: 3.92 s


In [147]:
%%time
gwr_compare = pd.DataFrame({'Actual # Cases': y_n.flatten(), 'Predicted # Cases': gwr_results.predictions.flatten()}, index = cases_ny.index)
gwr_compare['ratio'] = gwr_compare['Predicted # Cases']/gwr_compare['Actual # Cases']
gwr_compare['ratio'] = gwr_compare['ratio'].replace(np.inf, 1000)
gwr_compare[gwr_compare['ratio'] <= 0] = 0.01
gwr_compare
# gwr_compare.plot(kind='bar', figsize=(10,5))
# plt.show()

CPU times: user 16.5 ms, sys: 1.15 ms, total: 17.6 ms
Wall time: 4.26 ms


,Actual # Cases,Predicted # Cases,ratio
FIPS,,,
1001,17.00,45.819309,2.695253
1003,59.00,93.918141,1.591833
1005,9.00,88.122552,9.791395
1007,11.00,11.401303,1.036482
1009,12.00,12.596960,1.049747
...,...,...,...
56037,6.00,19.039969,3.173328
56039,50.00,79.740750,1.594815
56041,4.00,9.691534,2.422883


In [148]:
%%time
fig = go.Figure(
    go.Choroplethmapbox(
        geojson=counties1, locations=gwr_compare.index, 
        z=np.log(gwr_compare['ratio']),
        colorscale=[[0, 'blue'], [0.5,'white'], [1,'red']],
#         colorscale=[[0, 'blue'], [1,'red']],
        zmid = 0,
        zauto = True,
        marker_opacity=0.5, marker_line_width=0,
        ids = cases_ny['Combined_Key'],  
        name = 'Predicted / Actual # Cases',
        colorbar_thickness = 10,
        hoverinfo = 'text',
        text = cases_ny['Combined_Key'] + ' <br>Ratio: ' + gwr_compare['ratio'].round(3).astype('str'),
#         showlegend = True,
        showscale = True,
        colorbar = dict(
            title = "Ratio",
            titleside = 'top',
            tickmode = 'array',
            tickvals = [-2, -0.5, 0, 0.5, 2],
            ticktext = ['Underestimated', '<br>Slightly<br>Underestimated', 'As Estimated', 
                        'Slightly<br>Overestimated<br>', 'Overestimated'],
            ticks = 'inside',
            outlinewidth = 0
        )
    ))
fig.update_layout(mapbox_style="carto-positron",
                  mapbox_zoom=3, #mapbox_center = {"lat": 37.0902, "lon": -95.7129},)
                  mapbox_center={"lat": 40, "lon": -100},
                 )
fig.update_layout(margin={"r":10,"t":10,"l":10,"b":10})

fig.write_html('viz/gwr.html')

CPU times: user 2.89 s, sys: 44.5 ms, total: 2.94 s
Wall time: 2.49 s


In [150]:
gwr_param_col = ['Const']
gwr_param_col.extend(selected_vars)
gwr_param = pd.DataFrame(gwr_results.params, index = gwr_compare.index, columns = gwr_param_col)

In [151]:
gwr_param

,Const,POP12_SQMI,HSEHLD_1_F,HSEHLD_1_M,RENTER_OCC,HSE_UNITS,HOUSEHOLDS,ELDER,FEMALES,WHITE,...,MALES,FAMILIES,FHH_CHILD,MARHH_CHD,MULT_RACE,MHH_CHILD,ASIAN,HISPANIC,OTHER,AMERI_ES
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,9.576295,0.107301,-0.033769,-0.051939,-0.004975,0.001696,0.031219,-0.005912,-0.004384,0.000948,...,-0.005843,0.092572,-0.139953,-0.104063,-0.019430,-0.026534,0.027032,0.006095,-0.005660,-0.003591
1003,-101.194045,1.756876,-0.132600,0.330139,-0.011535,-0.003085,-0.004606,0.044806,-0.020634,-0.029803,...,-0.012125,0.110368,-0.450751,-0.007563,-0.182203,0.299274,-0.050708,0.198761,-0.382323,0.023980
1005,17.267844,-0.259616,-0.172687,0.079450,0.145293,0.011595,-0.111907,0.027810,0.006842,-0.020177,...,0.016234,0.054921,-0.179180,0.100200,-0.283981,0.081403,-0.004301,0.079235,-0.154932,-0.030617
1007,-0.429411,0.197661,-0.003955,0.012480,-0.012142,0.005903,0.004039,-0.003720,0.002944,0.002786,...,0.001857,0.041563,-0.041380,-0.038583,-0.011917,-0.049088,0.021786,0.008266,-0.009664,0.000853
1009,9.624683,-0.109668,0.050527,-0.043153,0.005713,0.010392,-0.016044,-0.008139,-0.004564,-0.000034,...,-0.005664,0.054594,-0.094792,-0.030632,0.026200,0.044093,-0.000387,-0.005615,0.005141,-0.024247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56037,-4.177774,0.026613,0.095167,-0.128746,-0.034048,0.013542,0.020136,-0.029456,0.061011,-0.023766,...,0.057614,0.053802,-0.157891,-0.091762,-0.178211,0.079284,-0.029639,0.013547,-0.038070,-0.015792
56039,-11.222062,0.615762,-0.052068,-0.113081,-0.043724,0.019878,0.102083,0.040223,0.066159,-0.034557,...,0.026173,-0.645958,0.544571,0.537002,-0.039575,0.833523,-0.003379,0.015553,-0.050598,-0.033994
56041,-8.667141,0.136479,0.093822,-0.150915,-0.015638,0.018551,0.048319,-0.007090,0.089810,-0.077970,...,0.071326,-0.187066,0.057361,0.187543,-0.135568,0.400458,-0.118241,0.004417,-0.077170,-0.070042


In [152]:
X_new_n[:,2]

array([2012., 8096., 1224., ...,  906.,  469.,  501.])

In [165]:
%%time
fig = go.Figure()

buttons = []
i = 0
for factor in selected_vars:
    fig.add_trace(
        dict(
            type='choroplethmapbox',
            geojson=counties1, locations=gwr_param.index, 
            z=gwr_param[factor],
            colorscale=[[0, 'blue'], [0.5,'white'], [1,'red']],
            zmid = gwr_param[factor].median(),
            zauto = True,
            marker_opacity=0.5, marker_line_width=0,
            ids = cases_ny['Combined_Key'],  
            name = 'Predicted / Actual # Cases',
            colorbar_thickness = 10,
            hoverinfo = 'text',
            text = 'Coefficient of ' + factor + ':<br>' + gwr_param[factor].astype(str),
            showscale = True,
            colorbar = dict(
                title = "Ratio",
                titleside = 'top',
                tickmode = 'array',
                ticks = 'inside',
                outlinewidth = 0
            )
        )
    )
    
    visible = [False] * len(selected_vars)
    visible[i] = True
    buttons.append({
        'label':factor,
        'method':'update',
        'args':[{
            'visible': visible,
        }]
    })
    
    i = i + 1

fig.data[0].visible=True    
    
fig.update_layout(mapbox_style="carto-positron",
                  mapbox_zoom=3, #mapbox_center = {"lat": 37.0902, "lon": -95.7129},)
                  mapbox_center={"lat": 40, "lon": -100},
                 )
fig.update_layout(margin={"r":10,"t":10,"l":10,"b":10})

fig.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=buttons,
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1, xanchor="left",
            y=1.1, yanchor="top"
        )
    ]
)

fig.write_html('viz/coeff.html')

CPU times: user 7.54 s, sys: 134 ms, total: 7.68 s
Wall time: 7.82 s
